# XGB Regression Model Prediction

#       

   The necesarily libries:

In [27]:
import numpy as np 
import pandas as pd 
import seaborn as sns 
import missingno as msno
import matplotlib.pyplot as plt
from sklearn import preprocessing



from sklearn.utils import check_array
from sklearn.model_selection import KFold
from sklearn.metrics import r2_score
from sklearn.metrics import median_absolute_error
import math

import warnings
warnings.filterwarnings("ignore")

In [28]:
data = pd.read_csv('encoded2.csv')

In [29]:
data

,car_name,year,mileage,fuel_type,seller_type,transmission_type,previous_owners,selling_price
0,84,2014.0,27000,2,0,1,0,3.35
1,87,2013.0,43000,1,0,1,0,4.75
2,66,2017.0,6900,2,0,1,0,7.25
3,90,2011.0,5200,2,0,1,0,2.85
4,86,2014.0,42450,1,0,1,0,4.60
...,...,...,...,...,...,...,...,...
251,67,2010.0,38000,2,0,1,0,3.25
252,63,2014.0,33019,2,0,1,0,3.75
253,67,2015.0,60076,1,0,1,0,8.55
254,65,2015.0,60000,2,0,1,0,4.00


   Defining the independent variables and dependent variable:

In [30]:
y=data.selling_price
X=data.drop(columns=['selling_price'])

Running and fitting the model:

In [31]:
from xgboost.sklearn import XGBRegressor
from sklearn.model_selection import GridSearchCV
parameters = {'nthread':[4], 
              'objective':['reg:linear'],
              'learning_rate': [.03, 0.05, .07], 
              'max_depth': [5, 6, 7],
              'min_child_weight': [4],
              'silent': [1],
              'subsample': [0.7],
              'colsample_bytree': [0.7],
              'n_estimators': [500]}

xgb1 = XGBRegressor()
xgb_grid = GridSearchCV(xgb1,
                        parameters,
                        cv = 2,
                        n_jobs = 5,
                        verbose=True)

Implementing a cross Valiadtion on the model:

In [32]:
from sklearn.model_selection import cross_val_score

reg_score= cross_val_score(xgb1, X,y, cv=10, scoring= 'neg_mean_absolute_error')
reg_score

array([-1.05671771, -0.8526869 , -1.2086212 , -0.25792876, -0.27672187,
       -0.26258915, -0.46808766, -1.00271775, -0.87932094, -0.87766886])

In [33]:
reg_score_n = -reg_score
reg_score_n

array([1.05671771, 0.8526869 , 1.2086212 , 0.25792876, 0.27672187,
       0.26258915, 0.46808766, 1.00271775, 0.87932094, 0.87766886])

Calculating the mean absolute error for all validation folds:

In [34]:
np.mean(reg_score_n)

0.7143060806246746

Comparison between the actual and predicted values:

In [35]:
from sklearn.model_selection import cross_val_predict
y_pred= cross_val_predict(xgb1, X, y, cv=10,  method= 'predict')


In [36]:
df= pd.DataFrame(data={'prediction' : y_pred, 'actual' :y})
df

,prediction,actual
0,5.002519,3.35
1,4.692098,4.75
2,7.851003,7.25
3,2.077850,2.85
4,5.134691,4.60
...,...,...
251,4.938484,3.25
252,3.720238,3.75
253,6.480326,8.55
254,4.037320,4.00
